# Collaboration and Competition using ddpg

---

In this notebook, I use the DDPG baseline and some variation of it to train multiples agents to play tennis in a Unity ML-Agents environment. This project correspond to the third project of the [Deep Reinforcement Learning Nanodegree](https://www.udacity.com/course/deep-reinforcement-learning-nanodegree--nd893) program.

## 1. Naive Approach
### Start the Environment
First, I imports necessary packages. If the code cell below returns an error, please revisit the project instructions to double-check that you have installed [Unity ML-Agents](https://github.com/Unity-Technologies/ml-agents/blob/master/docs/Installation.md) and [NumPy](http://www.numpy.org/).

In [1]:
from unityagents import UnityEnvironment
from unityagents import UnityEnvironment
import numpy as np
from collections import deque
import matplotlib.pyplot as plt
%matplotlib inline
import torch


Before running the next cell, check that the path for the environment is correct. Change the `file_name` parameter to match the location of the Unity environment that you downloaded before.

- **Mac**: `"path/to/Tennis.app"`
- **Windows** (x86): `"path/to/Tennis_Windows_x86/Tennis.exe"`
- **Windows** (x86_64): `"path/to/Tennis_Windows_x86_64/Tennis.exe"`
- **Linux** (x86): `"path/to/Tennis_Linux/Tennis.x86"`
- **Linux** (x86_64): `"path/to/Tennis_Linux/Tennis.x86_64"`
- **Linux** (x86, headless): `"path/to/Tennis_Linux_NoVis/Tennis.x86"`
- **Linux** (x86_64, headless): `"path/to/Tennis_Linux_NoVis/Tennis.x86_64"`

In [2]:
env = UnityEnvironment(file_name="/home/raphe/Cours/Nanodegree_DRL/Tennis_Linux/Tennis.x86_64")

INFO:unityagents:
'Academy' started successfully!
Unity Academy name: Academy
        Number of Brains: 1
        Number of External Brains : 1
        Lesson number : 0
        Reset Parameters :
		
Unity brain name: TennisBrain
        Number of Visual Observations (per agent): 0
        Vector Observation space type: continuous
        Vector Observation space size (per agent): 8
        Number of stacked Vector Observation: 3
        Vector Action space type: continuous
        Vector Action space size (per agent): 2
        Vector Action descriptions: , 


Environments contain **_brains_** which are responsible for deciding the actions of their associated agents. Here we check for the first brain available, and set it as the default brain we will be controlling from Python.

In [3]:
# Here we take the brain name  and set it as the default
# Environments contain brains which are responsible for deciding the actions of their associated agents
# We use 2 agents. Depending on the algorythm used to solve the environement
# the agents may collaborate or not. But from a technical point of view one could say that they are indepandant
# as they dont evolve in the same space.
brain_name = env.brain_names[0]
brain = env.brains[brain_name]

### Examine the State and Action Spaces

As I stated in README.MD, the goal of the two agents is to bounce a ball over a net using a racket as long as possible. If the agent hits the ball over the net, it receives a reward of +0.1. But, if the agent lets the ball hit the ground or hit it oustide of the table, it receives a reward of -0.01.  Thus, the goal of each agent is to keep the ball in play.

The observation space consists of 8 variables corresponding to the position and velocity of the ball and racket.
Each agent has only two continuous actions available:
- moving toward or away the net
- jumping. 

In [4]:
# reset the environment
env_info = env.reset(train_mode=True)[brain_name]

# number of agents 
num_agents = len(env_info.agents)
print('Number of agents:', num_agents)

# size of each action
action_size = brain.vector_action_space_size
print('Size of each action:', action_size)

# examine the state space 
states = env_info.vector_observations
state_size = states.shape[1]
print('There are {} agents. Each observes a state with length: {}'.format(states.shape[0], state_size))
print('The state for the first agent looks like:', states[0])

Number of agents: 2
Size of each action: 2
There are 2 agents. Each observes a state with length: 24
The state for the first agent looks like: [ 0.          0.          0.          0.          0.          0.
  0.          0.          0.          0.          0.          0.
  0.          0.          0.          0.         -6.65278625 -1.5
 -0.          0.          6.83172083  6.         -0.          0.        ]


### Take Random Actions in the Environment

In the next code cell, I use the Python API to control the agents and receive feedback from the environment.

As one can see, the comportement of each agent and his performance are really bad. 
This is due to the fact that the agent has not yet learned how to play tennis. I use a random function to define the actions values. 

## Training

In [5]:
import numpy as np

import torch
import torch.nn as nn
import torch.nn.functional as F

def hidden_init(layer):
    fan_in = layer.weight.data.size()[0]
    lim = 1. / np.sqrt(fan_in)
    return (-lim, lim)

class Actor(nn.Module):
    """Actor (Policy) Model."""

    def __init__(self, state_size, action_size, seed=0, fc1_units=256,fc2_units=128):
        """Initialize parameters and build model.
        Params
        ======
            state_size (int): Dimension of each state
            action_size (int): Dimension of each action
            seed (int): Random seed
            fc1_units (int): Number of nodes in first hidden layer
            fc2_units (int): Number of nodes in second hidden layer
        """
        super(Actor, self).__init__()
        self.seed = torch.manual_seed(seed)
        self.fc1 = nn.Linear(state_size,fc1_units)
        self.fc2 = nn.Linear(fc1_units, fc2_units)
        self.fc3 = nn.Linear(fc2_units, action_size)
        self.reset_parameters()

    def reset_parameters(self):
        self.fc1.weight.data.uniform_(*hidden_init(self.fc1))
        self.fc2.weight.data.uniform_(*hidden_init(self.fc2))
        self.fc3.weight.data.uniform_(-3e-3, 3e-3)

    def forward(self, state):
        """Build an actor (policy) network that maps states -> actions."""
        x = F.relu(self.fc1(state))
        x = F.relu(self.fc2(x))
        return torch.tanh(self.fc3(x))


class Critic(nn.Module):
    """Critic (Value) Model."""

    def __init__(self, states_size, actions_size, seed=0, fc1_units=256, fc2_units=128):
        """Initialize parameters and build model.
        Params
        ======
            state_size (int): Dimension of each state
            action_size (int): Dimension of each action
            seed (int): Random seed
            fcs1_units (int): Number of nodes in the first hidden layer
            fc2_units (int): Number of nodes in the second hidden layer
        """
        super(Critic, self).__init__()
        self.seed = torch.manual_seed(seed)
        self.fc1 = nn.Linear(states_size, fc1_units)
        self.fc2 = nn.Linear(fc1_units+actions_size, fc2_units)
        #self.fc2 = nn.Linear(fc1_units, fc2_units)

        self.fc3 = nn.Linear(fc2_units, 1)
        self.reset_parameters()

    def reset_parameters(self):
        self.fc1.weight.data.uniform_(*hidden_init(self.fc1))
        self.fc2.weight.data.uniform_(*hidden_init(self.fc2))
        self.fc3.weight.data.uniform_(-3e-3, 3e-3)
        
    def forward(self, states, actions):
        """Build a critic (value) network that maps (state, action) pairs -> Q-values."""
        xs = states
        xa = actions
        xs = F.relu(self.fc1(xs))
        xs = torch.cat((xs,xa), dim=1)
        x  = F.relu(self.fc2(xs))  
        return self.fc3(x)



### Parameters

In [6]:
import numpy as np
import random
import copy
from collections import namedtuple, deque


import torch
import torch.nn.functional as F
import torch.optim as optim

BUFFER_SIZE = int(2e5)  # replay buffer size
BATCH_SIZE = 128        # minibatch size
GAMMA = 0.99            # discount factor
TAU = 1e-3              # for soft update of target parameters
LR_ACTOR = 1e-4         # learning rate of the actor 
LR_CRITIC = 1e-4        # learning rate of the critic
WEIGHT_DECAY = 0.001     # L2 weight decay
UPDATE_EVERY=1
device = torch.device("cuda:0" if torch.cuda.is_available() else "cpu")

### Class Noise

In [7]:
class OUNoise:
    """Ornstein-Uhlenbeck process."""

    def __init__(self, size, seed, mu=0., theta=0.15, sigma=0.2):
        """Initialize parameters and noise process."""
        self.mu = mu * np.ones(size)
        self.theta = theta
        self.sigma = sigma
#        random.seed(seed)
        np.random.seed(seed)
        self.reset()

    def reset(self):
        """Reset the internal state (= noise) to mean (mu)."""
        self.state = copy.copy(self.mu)

    def sample(self):
        """Update internal state and return it as a noise sample."""
        x = self.state
# default : Uniform
#        dx = self.theta * (self.mu - x) + self.sigma * np.array([random.random() for i in range(len(x))])
# DDPG paper: normal distribution
        dx = self.theta * (self.mu - x) + self.sigma * np.array([np.random.randn() for i in range(len(x))])

        self.state = x + dx
        return self.state


### Replay buffer

In [8]:
class ReplayBuffer:
    """Fixed-size buffer to store experience tuples."""

    def __init__(self, buffer_size, batch_size, seed):
        """Initialize a ReplayBuffer object.
        Params
        ======
            buffer_size (int): maximum size of buffer
            batch_size (int): size of each training batch
        """

        self.memory = deque(maxlen=buffer_size)  # internal memory (deque)
        self.batch_size = batch_size
        self.experience = namedtuple("Experience", field_names=["state", "action", "reward", "next_state", "done"])
        random.seed(seed)
    
    def add(self, state, action, reward, next_state, done):
        """Add a new experience to memory."""
        e = self.experience(state, action, reward, next_state, done)
        self.memory.append(e)
    
    def sample(self):
        """Randomly sample a batch of experiences from memory."""
        experiences = random.sample(self.memory, k=self.batch_size)
        states = torch.from_numpy(np.vstack([e.state for e in experiences if e is not None])).float().to(device)
        actions = torch.from_numpy(np.vstack([e.action for e in experiences if e is not None])).float().to(device)
        rewards = torch.from_numpy(np.vstack([e.reward for e in experiences if e is not None])).float().to(device)
        next_states = torch.from_numpy(np.vstack([e.next_state for e in experiences if e is not None])).float().to(device)
        dones = torch.from_numpy(np.vstack([e.done for e in experiences if e is not None]).astype(np.uint8)).float().to(device)

        return states, actions, rewards, next_states, dones

    def __len__(self):
        """Return the current size of internal memory."""
        return len(self.memory)


### Class agent 

In [9]:
class Agent():
    """Interacts with and learns from the environment."""
    
    def __init__(self, state_size, action_size,nb_agent=1 ,random_seed=0):
        """Initialize an Agent object.
        
        Params
        ======
            state_size (int): dimension of each state
            action_size (int): dimension of each action
            random_seed (int): random seed
        """
        self.state_size = state_size
        self.action_size = action_size
        self.states_size = state_size*nb_agent
        self.actions_size = action_size*nb_agent       
       # self.seed = random.seed(random_seed)
        self.rate=1
        # Actor Network (w/ Target Network)
        self.actor_local= []
        self.actor_target= []
        self.actor_optimizer= []
        self.critic_local= []
        self.critic_target= []
        self.critic_optimizer= []  
        for i in range(nb_agent):         
            self.actor_local.append(Actor(state_size, action_size).to(device))
            self.actor_target.append(Actor(state_size, action_size).to(device))
            self.actor_optimizer.append( optim.Adam(self.actor_local[i].parameters(), lr=LR_ACTOR))

        # Critic Network (w/ Target Network)
            self.critic_local.append(Critic(self.states_size, self.actions_size).to(device))
            self.critic_target.append( Critic(self.states_size, self.actions_size).to(device))
            self.critic_optimizer.append(optim.Adam(self.critic_local[i].parameters(), lr=LR_CRITIC, weight_decay=WEIGHT_DECAY))

        # Noise process
        self.noise = OUNoise(action_size, random_seed)

        # Replay memory
        self.memory = ReplayBuffer( BUFFER_SIZE, BATCH_SIZE, random_seed)
        self.t_step = 0
        
   

    def act(self, state, add_noise=True,rate=0.99,ind=0):
        """Returns actions for given state as per current policy."""
        state = torch.from_numpy(state).float().to(device)
        self.actor_local[ind].eval()
        self.rate=rate*self.rate
        with torch.no_grad():
            action = self.actor_local[ind](state).cpu().data.numpy()
        self.actor_local[ind].train()
        if add_noise:
            action += self.noise.sample()*self.rate
        return np.clip(action, -1, 1)

    def reset(self):
        self.noise.reset()

    def soft_update(self, local_model, target_model, tau):
        """Soft update model parameters.
        θ_target = τ*θ_local + (1 - τ)*θ_target

        Params
        ======
            local_model: PyTorch model (weights will be copied from)
            target_model: PyTorch model (weights will be copied to)
            tau (float): interpolation parameter 
        """
        for target_param, local_param in zip(target_model.parameters(), local_model.parameters()):
            target_param.data.copy_(tau*local_param.data + (1.0-tau)*target_param.data)
        
        
    def learn(self,states,actions,rewards,dones,next_states,next_actions,actions_local, gamma,ind=0,nb_agent=1):
 
        """Update policy and value parameters using given batch of experience tuples.
        Q_targets = r + γ * critic_target(next_state, actor_target(next_state))
        where:
            actor_target(state) -> action
            critic_target(state, action) -> Q-value

        Params
        ======
            experiences (Tuple[torch.Tensor]): tuple of (s, a, r, s', done) tuples 
            gamma (float): discount factor
        """


        # ---------------------------- update critic ---------------------------- #
        # get predicted next-state actions and Q values from target models
        self.critic_optimizer[ind].zero_grad()
        
        #with torch.no_grad():
        q_targets_next = self.critic_target[ind](next_states, next_actions)
        # compute Q targets for current states (y_i)
        q_expected = self.critic_local[ind](states, actions)
        # q_targets = reward of this timestep + discount * Q(st+1,at+1) from target network
        #print("\nQtarget:\n",rewards[:,ind,:] + (gamma * q_targets_next * (1 - dones[:,ind,:])))
        q_targets = rewards[:,ind,:] + (gamma * q_targets_next * (1 - dones[:,ind,:]))
        # compute critic loss
        critic_loss = F.mse_loss(q_expected, q_targets.detach())
        # minimize loss
        critic_loss.backward()
        self.critic_optimizer[ind].step()

        # ---------------------------- update actor ---------------------------- #
        # compute actor loss
        self.actor_optimizer[ind].zero_grad()
        # detach actions from other agents
        actor_loss = -self.critic_local[ind](states, actions_local).mean()
        # minimize loss
        actor_loss.backward()
        self.actor_optimizer[ind].step()

        #----------------------- update target networks ----------------------- #
        self.soft_update(self.critic_local[ind], self.critic_target[ind], TAU)
        
        for target_param, local_param in zip(self.critic_target[ind].parameters(), self.critic_local[ind].parameters()):
            target_param.data.copy_(tau*local_param.data + (1.0-tau)*target_param.data)
            
        self.soft_update(self.actor_local[ind], self.actor_target[ind], TAU)                     
        return critic_loss,actor_loss

    def step(self, states, actions, rewards, next_states, dones,nb_agent):
        """Save experience in replay memory, and use random sample from buffer to learn."""
        # Save experience / reward
        # self.memory.add(state, action, reward, next_state, done)
        # Save experience / reward
        self.memory.add(states, actions, rewards, next_states, dones)  
        critic=0
        actor=0
        # Learn, if enough samples are available in memory
        # Learn every UPDATE_EVERY time steps.
        self.t_step = (self.t_step + 1) % UPDATE_EVERY
        if self.t_step == 0:
            if len(self.memory) > BATCH_SIZE:
                # each agent uses it's own actor to calculate next_actions
                for ind in range(nb_agent):
                    # each agent learns from it's experience sample
                    states, actions, rewards, next_states, dones = self.memory.sample()
 
                    # reshape
                    states_v    = states.view(BATCH_SIZE,nb_agent,24).squeeze(1)
                    next_states_v= next_states.view(BATCH_SIZE,nb_agent,24).squeeze(1)     
                    actions_v   = actions.view(BATCH_SIZE,nb_agent,2).squeeze(1)
                    rewards_v   = rewards.view(BATCH_SIZE,nb_agent,1).squeeze(1)
                    dones_v     = dones.view(BATCH_SIZE,nb_agent,1).squeeze(1)
                    # Next action
                    #print("\nactions\n:",actions)
                    all_next_actions = []
                    all_next_states = []
                    all_states =[]
                    all_actions = []
                    for i in range(nb_agent):
                        all_next_actions.append(self.actor_target[i](next_states_v[:,i,:]))
                        all_next_states.append((next_states_v[:,i,:]))
                        all_states.append((states_v[:,i,:]))
                        all_actions.append((actions_v[:,i,:]))
                   # Action with i = action_i
                    actions_next = torch.cat(all_next_actions, dim=1).to(device)
                    states_next = torch.cat(all_next_states, dim=1).to(device)
                    states_s = torch.cat(all_states, dim=1).to(device)
                    actions_s= torch.cat(all_actions, dim=1).to(device)
                    all_actions = []
                    for i in range(nb_agent):
                        if(ind==i):
                            action = agent.actor_local[ind](states_v[:,ind,:]).detach()

                        else:
                            action = actions_v[:,i,:]
                        all_actions.append(action)
                    
                    all_actions_local=torch.cat(all_actions, dim=1).to(device)
 
                #    print("\nact_next\n",all_actions_local)
                 #   print("\nsta_next\n",states_s)
                   # print("cat\n",torch.cat(states_next,actions_next,dim=1))
                 #   print("\ndones   \n",dones_v[:,ind,:]*q_targets_next)
                    critic_loss,actor_loss= self.learn(states_s.detach(),\
                               actions_s.detach(),\
                               rewards_v.detach(),\
                               dones_v.detach(),\
                               states_next.detach(),\
                               actions_next.detach(),\
                               all_actions_local,\
                               GAMMA,\
                               ind,\
                               nb_agent)                    
                    critic=critic+critic_loss
                    actor=actor+actor_loss
                return critic/nb_agent,actor/nb_agent
        return 0,0

In [10]:
agent =Agent(state_size=state_size, action_size=action_size,nb_agent=2, random_seed=10)

In [11]:
def multi_ddpg(env, env_info, state_size, action_size, brain_name,nb_agent, agent,n_episodes=4000, max_t=1000):
    scores_deque = deque(maxlen=100)
    scores = []

    for i_episode in range(1, n_episodes):
        env_info = env.reset(train_mode=True)[brain_name]  # reset the environment
        states = env_info.vector_observations  # get the current state (for each agent)
        agent.reset() # reset the noise added to the state. Makes the training more robust.
        score=np.zeros(num_agents)  # initialize the score (for each agent)
        for t in range(max_t):
           # actions = agent.act(states)
            actions = [agent.act(states[i],rate=0.95,ind=i) for i in range(nb_agent)] # get action from each agent based on the current state
            env_info = env.step(actions)[brain_name]  # update environment informations with the actions of each agent
            next_states = env_info.vector_observations  # get next state (for each agent)
            rewards = env_info.rewards  # get reward (for each agent)
            score = score+rewards  # update the score for each agent
            dones = env_info.local_done  # see if episode finished
            # agent[i].step: add (states,actions,rewards,next_states) to replay buffer of each agent 
            # train the actor critic Neural Network of each agent
            # each agent share the same information
            if(t==0 and any(dones)):
                if(dones[0]):
                    rewards[0]=-2.
                else:
                    rewards[1]=-2.
            critic,actor=agent.step(states, actions, rewards, next_states, dones,nb_agent) 
            states = next_states # roll over the state to next time step
 
            if any(dones):
                break
                
        scores.append(np.max(score)) # save the best agent score for display
        scores_deque.append(np.max(score)) # save the best agent score into the windows for convergence checking
      #  print('Episode {}\t Average Score: {:.4E}\t: reward:{} critic:{:.4E}\t actor:{:.4E}\t'.format(i_episode, np.mean(scores_deque), np.max(score),critic,actor), end="")
        if i_episode>100 and np.mean(scores_deque)>0.5: # check if env is solved
            print("envionment solved")
            [torch.save(agent.actor_local[i].state_dict(), 'multi_checkpoint_actor'+str(i)+'.pth') for i in range(num_agents)] # save actor weights for each agents
            [torch.save(agent.critic_local[i].state_dict(), 'multi_checkpoint_critic'+str(i)+'.pth') for i in range(num_agents)] # save critic weights for each agents
            return scores
        
        if i_episode%10 ==0:
            print('E {}\t A Score: {:.4E}\t: r:{}\n'.format(i_episode, np.mean(scores_deque), np.max(score),critic,actor), end="")

        #    [torch.save(agent.actor_local[i].state_dict(), 'multi_checkpoint_actor'+str(i)+'.pth') for i in range(num_agents)] # save actor weights for each agents
        #    [torch.save(agent.critic_local[i].state_dict(), 'multi_checkpoint_critic'+str(i)+'.pth') for i in range(num_agents)] # save critic weights for each agents
   
    return scores

In [ ]:
scores = multi_ddpg(env, env_info, state_size, action_size, brain_name,num_agents, agent)

E 10	 A Score: 0.0000E+00	: r:0.0
E 20	 A Score: 4.5000E-03	: r:0.0
E 30	 A Score: 3.0000E-03	: r:0.0
E 40	 A Score: 2.2500E-03	: r:0.0
E 50	 A Score: 1.8000E-03	: r:0.0
E 60	 A Score: 1.5000E-03	: r:0.0
E 70	 A Score: 4.1429E-03	: r:0.0
E 80	 A Score: 4.7500E-03	: r:0.0
E 90	 A Score: 4.2222E-03	: r:0.0
E 100	 A Score: 3.8000E-03	: r:0.0
E 110	 A Score: 3.8000E-03	: r:0.0
E 120	 A Score: 2.9000E-03	: r:0.0
E 130	 A Score: 2.9000E-03	: r:0.0
E 140	 A Score: 4.8000E-03	: r:0.0
E 150	 A Score: 4.8000E-03	: r:0.0
E 160	 A Score: 4.8000E-03	: r:0.0
E 170	 A Score: 3.8000E-03	: r:0.0
E 180	 A Score: 2.9000E-03	: r:0.0
E 190	 A Score: 2.9000E-03	: r:0.0
E 200	 A Score: 2.9000E-03	: r:0.0
E 210	 A Score: 2.9000E-03	: r:0.0
E 220	 A Score: 5.8000E-03	: r:0.10000000149011612
E 230	 A Score: 5.8000E-03	: r:0.0
E 240	 A Score: 3.9000E-03	: r:0.0
E 250	 A Score: 3.9000E-03	: r:0.0
E 260	 A Score: 3.9000E-03	: r:0.0
E 270	 A Score: 3.9000E-03	: r:0.0
E 280	 A Score: 3.9000E-03	: r:0.0
E 290	 A Scor

In [ ]:
#Episode 1972	Average Score: 0.5029000074975193	Score: 1.0000000149011612envionment solved

#### Display results for training

##### Score

In [ ]:
fig = plt.figure()
ax = fig.add_subplot(111)
plt.plot(np.arange(1, len(scores)+1), scores)
plt.ylabel('Score')
plt.xlabel('Episode #')
plt.show()

#### End of training

In [ ]:
#env.close()

### 2. Test

#### Load libraries

In [ ]:
from unityagents import UnityEnvironment
import numpy as np
from collections import deque
import matplotlib.pyplot as plt
%matplotlib inline
import torch
 

#### Define testing function

In [ ]:
def ddpg_Test(env,brain_name,num_agents,agent,file_name_actor,file_name_critic,n_episodes=6000, max_t=1000):
    """ Visualize agent using saved checkpoint. """
    # load saved weights
    agent.actor_local.load_state_dict(torch.load(file_name_actor))
    agent.critic_local.load_state_dict(torch.load(file_name_critic)) 
    scores = []  # list containing scores from each episode
 
    for i_episode in range(1, n_episodes):
        env_info = env.reset(train_mode=False)[brain_name]  # environment informations
        states = env_info.vector_observations 
        agent.reset()
        score = np.zeros(num_agents) 
        for t in range(max_t):
            actions = agent.act(states,rate=0.9999) # get action using the DDPG algorythme (for each agent)
            env_info = env.step(actions)[brain_name]  # update environment informations with the actions of each agent
            next_states = env_info.vector_observations  # get next state (for each agent)
            rewards = env_info.rewards  # get reward (for each agent)
            dones = env_info.local_done  # see if episode finished
            states = next_states  # roll over the state to next time step
            score =score+ rewards  # update the score for each agent
            if any(dones):  # see if episode has finished
                break
        print('\rEpisode {}\tAverage Score: {}, touch: {}'.format(i_episode, np.max(score),t), end="")
        scores.append(np.max(score)) # save the best score between both agents

    return scores



In [ ]:
def double_ddpg_Test(env,brain_name,num_agents,agent,file_name_actor,file_name_critic,n_episodes=6000, max_t=1000):
    """ Visualize agent using saved checkpoint. """
    # load saved weights
    for i in range(num_agents):
        agent.actor_local[i].load_state_dict(torch.load(file_name_actor[i]))
        agent.critic_local[i].load_state_dict(torch.load(file_name_critic[i])) 
        
    scores = []                        # list containing scores from each episode
    score = 0
 
    for i_episode in range(1, n_episodes):
        env_info = env.reset(train_mode=False)[brain_name]
        states = env_info.vector_observations 
        agent.reset() 
        score = np.zeros(num_agents) 
        for t in range(max_t):
            actions = [agent.act(states[i],rate=0.95,ind=i) for i in range(nb_agent)] # get action from each agent based on the current state
            env_info = env.step(actions)[brain_name] 
            next_states = env_info.vector_observations         # get next state (for each agent)
            rewards = env_info.rewards                         # get reward (for each agent)
            dones = env_info.local_done                        # see if episode finished
            #agent.step(states, actions, rewards, next_states, dones,num_agents)
            states = next_states
            score =score+ rewards
            if any(dones):
                break
        scores.append(np.max(score)) # save the best score between both agents
    print('\rEpisode {}\tAverage Score: {}'.format(i_episode, np.max(score)), end="")
    return scores

#### Load environment

In [ ]:
#env = UnityEnvironment(file_name="/home/raphe/Cours/Nanodegree_DRL/Tennis_Linux/Tennis.x86_64")

In [ ]:
# reset the environment
env_info = env.reset(train_mode=False)[brain_name]

# number of agents 
num_agents = len(env_info.agents)
print('Number of agents:', num_agents)

# size of each action
action_size = brain.vector_action_space_size
print('Size of each action:', action_size)

# examine the state space 
states = env_info.vector_observations
state_size = states.shape[1]
print('There are {} agents. Each observes a state with length: {}'.format(states.shape[0], state_size))
print('The state for the first agent looks like:', states[0])
nb_agent=2

In [ ]:
agent =Agent(state_size=state_size, action_size=action_size,nb_agent=2, random_seed=10)

#### Define the weights

In [ ]:
file_name_actor=[]
file_name_critic=[]
for i in range(num_agents):
    file_name_actor.append('multi_checkpoint_actor'+str(i)+'.pth')
    file_name_critic.append('multi_checkpoint_critic'+str(i)+'.pth')

#### Load the weight, start testing

In [ ]:
scores=double_ddpg_Test(env,brain_name,num_agents,agent,file_name_actor,file_name_critic ,n_episodes=20,max_t=100)

 #### Display score

In [ ]:
fig = plt.figure()
ax = fig.add_subplot(111)
plt.plot(np.arange(1, len(scores)+1), scores)
plt.ylabel('Score')
plt.xlabel('Episode #')
plt.show()

#### End of testing

In [ ]:
env.close()